In [1]:
import numpy as np 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from src.data.loaddata import load_data
import os
import matplotlib.pyplot as plt
import pickle
from src.data.Undersample import equnder_sample,random_under_sample
from src.data.eval import evaluate_model
# <保存预处理数据，避免相同预处理重复执行> -----------------------
PROJ_TAG = 'c23_wosw_basic' # 项目标签
USE_NANSEIZURE_DATA = False # 是否使用不含癫痫发作的edf数据
USE_SAVED_DATA = False # 是否使用保存的预处理数据
SAVE_PROCESSED_DATA = False # 是否保存预处理后的数据

# <平衡数据处理> ----------------------------------------------
UNDER_SAMPLING_RATIO = 0.5 # 在过采样前保留多少比例的多数类样本
OVER_SAMPLING_RATIO = 0.5 # 若为1，则最后少数类样本数=多数类样本数
RAMDOM_UNDER_SAMPLING = True # 是否进行随机欠采样，False则进行等距欠采样
USE_SWIN = False # 是否使用滑窗丰富发作样本
WINDOW_NUM = 2 

In [2]:
def get_save_path(base_path, file_id, proj_tag):
    save_dir = os.path.join(base_path, proj_tag)
    os.makedirs(save_dir, exist_ok=True)  # 确保目录存在
    return os.path.join(save_dir, f"data_subject_{file_id}.pkl")

# <读取含发作事件的数据> ----------------------------------------------------------
seizure_base_path = "data"
seizure_file_ids = [1,2]

all_X_ws = []
all_y_ws = []

for file_id in seizure_file_ids:
    data_file = get_save_path(seizure_base_path, file_id, PROJ_TAG)
    if USE_SAVED_DATA and os.path.exists(data_file):
        print(f"Loading data from {data_file} for subject {file_id}...")
        with open(data_file, 'rb') as f:
            X, y = pickle.load(f)
    else:
        print(f"Data file not found, loading and saving data for subject {file_id}...")
        X, y = load_data(file_id, seizure_base_path)
        if SAVE_PROCESSED_DATA:
            with open(data_file, 'wb') as f:
                pickle.dump((X, y), f)
            print(f"Data saved to {data_file}.")
    all_X_ws.append(X)
    all_y_ws.append(y)

# <读取不含发作事件的数据> --------------------------------------------------------
nonseizure_base_path = "data_wos"
nonseizure_file_ids = [1, 2]

all_X_wos = []
all_y_wos = []

for file_id in nonseizure_file_ids:
    if USE_NANSEIZURE_DATA:
        data_file = get_save_path(nonseizure_base_path, file_id, PROJ_TAG)
        if USE_SAVED_DATA and os.path.exists(data_file):
            print(f"Loading data from {data_file} for subject {file_id}...")
            with open(data_file, 'rb') as f:
                X, y = pickle.load(f)
        else:
            print(f"Data file not found, loading and saving data for subject {file_id}...")
            X, y = load_data(file_id, nonseizure_base_path)
            if SAVE_PROCESSED_DATA:
                with open(data_file, 'wb') as f:
                    pickle.dump((X, y), f)
                print(f"Data saved to {data_file}.")
        all_X_wos.append(X)
        all_y_wos.append(y)

# <合并数据> -----------------------------------------------
if USE_NANSEIZURE_DATA:
    all_X = np.vstack(all_X_ws + all_X_wos)
    all_y = np.concatenate(all_y_ws + all_y_wos)
else:
    all_X = np.vstack(all_X_ws)
    all_y = np.concatenate(all_y_ws)

all_X = all_X.reshape(-1, all_X.shape[-1])
all_y = all_y.reshape(-1)
print("总数据：", all_X.shape, all_y.shape)
print("原始发作数据量：", np.sum(all_y == 1))
print("原始发作百分比：", np.sum(all_y == 1) / all_y.shape[0])

Data file not found, loading and saving data for subject 1...
Data file not found, loading and saving data for subject 2...


ValueError: cannot reshape array of size 0 into shape (0)

In [ ]:
# <读取含发作事件的数据> ----------------------------------------------------------
seizure_base_path = "data"
seizure_file_ids = [1, 21, 5, 14]

all_X_ws = []
all_y_ws = []

for file_id in seizure_file_ids:
    if USE_SAVED_DATA:
        data_file = f"{seizure_base_path}/data_subject_{file_id}.pkl"
        if os.path.exists(data_file):
            print(f"Loading data from {data_file} for subject {file_id}...")
            with open(data_file, 'rb') as f:
                X, y = pickle.load(f)
        else:
            print(f"Data file not found, loading and saving data for subject {file_id}...")
            X, y = load_data(file_id, seizure_base_path)
            if SAVE_PROCESSED_DATA:
                with open(data_file, 'wb') as f:
                    pickle.dump((X, y), f)
                print(f"Data saved to {data_file}.")
    else:
        X, y = load_data(file_id, seizure_base_path)
    all_X_ws.append(X)
    all_y_ws.append(y)
# <读取不含发作事件的数据> --------------------------------------------------------
nonseizure_base_path = "data_wos"
nonseizure_file_ids = [1, 2, 3]

all_X_wos = []
all_y_wos = []

for file_id in nonseizure_file_ids:
    if USE_NANSEIZURE_DATA:
        if USE_SAVED_DATA:
            data_file = f"{nonseizure_base_path}/data_subject_{file_id}.pkl"
            if os.path.exists(data_file):
                print(f"Loading data from {data_file} for subject {file_id}...")
                with open(data_file, 'rb') as f:
                    X, y = pickle.load(f)
            else:
                print(f"Data file not found, loading and saving data for subject {file_id}...")
                X, y = load_data(file_id, nonseizure_base_path)

                if SAVE_PROCESSED_DATA:
                    with open(data_file, 'wb') as f:
                        pickle.dump((X, y), f)
                    print(f"Data saved to {data_file}.")
        else:
            X, y = load_data(file_id, nonseizure_base_path)
        all_X_wos.append(X)
        all_y_wos.append(y)
# <合并数据> -----------------------------------------------
if USE_NANSEIZURE_DATA:
    all_X = np.vstack(all_X_ws + all_X_wos)
    all_y = np.concatenate(all_y_ws + all_y_wos)
else:
    all_X = np.vstack(all_X_ws)
    all_y = np.concatenate(all_y_ws)
all_X = all_X.reshape(-1, all_X.shape[-1])
all_y = all_y.reshape(-1)
print("总数据：", all_X.shape, all_y.shape)
print("原始发作数据量：", np.sum(all_y == 1))
print("原始发作百分比：", np.sum(all_y == 1) / all_y.shape[0])

In [ ]:
# <分离二元标签> ----------------------------------
def split_data(all_X, all_y):
    # 确保 all_y 是 NumPy 数组
    all_y = np.array(all_y)
    
    # 获取标签为 1 和 0 的样本
    one_indices = all_y == 1
    zero_indices = all_y == 0

    one_X = all_X[one_indices]
    one_y = all_y[one_indices]
    zero_X = all_X[zero_indices]
    zero_y = all_y[zero_indices]
    
    return one_X, one_y, zero_X, zero_y

one_X, one_y, zero_X, zero_y = split_data(all_X, all_y)

In [ ]:
# <欠采样> ----------------------------------------------
under_X, under_y = zero_X, zero_y
if UNDER_SAMPLING_RATIO > 0:
    if RAMDOM_UNDER_SAMPLING:
        under_X, under_y = random_under_sample(zero_X, zero_y, UNDER_SAMPLING_RATIO)
    else:
        under_X, under_y = equnder_sample(zero_X, zero_y, UNDER_SAMPLING_RATIO)
    print("欠采样后数据：", under_X.shape, under_y.shape)
# <合并数据> ----------------------------------------------
all_X = np.vstack([one_X, under_X])
all_y = np.concatenate([one_y, under_y])

# <过采样> ----------------------------------------------
# 过采样后比例 = 1 / (1 + 1/OVER_SAMPLING_RATIO)
if OVER_SAMPLING_RATIO > 0:
    smote = SMOTE(sampling_strategy=OVER_SAMPLING_RATIO)
    over_X, over_y = smote.fit_resample(all_X, all_y)
    print("过采样后数据：", over_X.shape, over_y.shape)
    print("过采样后发作数据量：", np.sum(over_y == 1))
    print("合成数据比例：", (np.sum(over_y == 1) - np.sum(one_y)) / np.sum(over_y == 1))
    print("过采样后发作百分比：", np.sum(over_y == 1) / over_y.shape[0])


In [ ]:
# <创建测试数据> ----------------------------------------------
base_path = "data"
file_ids = [16,3]
test_X = []
test_y = []
for file_id in file_ids:
    if USE_SAVED_DATA:
        data_file = f"{base_path}/data_subject_{file_id}.pkl"
        if os.path.exists(data_file):
            print(f"Loading data from {data_file} for subject {file_id}...")
            with open(data_file, 'rb') as f:
                X, y = pickle.load(f)
        else:
            print(f"Data file not found, loading and saving data for subject {file_id}...")
            X, y = load_data(file_id, base_path)
            if SAVE_PROCESSED_DATA:
                with open(data_file, 'wb') as f:
                    pickle.dump((X, y), f)
                print(f"Data saved to {data_file}.")
    else:
        X, y = load_data(file_id, base_path)
    test_X.append(X)
    test_y.append(y)
test_X = np.vstack(test_X)
test_y = np.concatenate(test_y)
test_X = test_X.reshape(-1, test_X.shape[-1])
test_y = test_y.reshape(-1)
print("测试数据：", test_X.shape, test_y.shape)
print("测试发作数据量：", np.sum(test_y == 1))
print("测试发作百分比：", np.sum(test_y == 1) / test_y.shape[0])

### 训练相关

In [ ]:
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(all_X, all_y, test_size=0.2, random_state=42, stratify=all_y)
# 训练随机森林分类器
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(train_X, train_y)
# 预测
# 输出y的正例数量
# 统计1的数量
val_y_pred = clf.predict(val_X)
evaluate_model(val_y, val_y_pred)

test_y_pred = clf.predict(test_X)
evaluate_model(test_y, test_y_pred)


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
# min-max标准化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
all_X = scaler.fit_transform(all_X)
test_X = scaler.transform(test_X)

# 切分数据
train_X, val_X, train_y, val_y = train_test_split(all_X, all_y, test_size=0.2, random_state=42, stratify=all_y)

# 训练SVM分类器
clf = SVC(random_state=0)  # 你可以根据需要调整其他参数，如kernel等
clf.fit(train_X, train_y)

# 预测
val_y_pred = clf.predict(val_X)
evaluate_model(val_y, val_y_pred)

test_y_pred = clf.predict(test_X)
evaluate_model(test_y, test_y_pred)


In [ ]:
# 分割处理后的数据集
all_X_2d = all_X.reshape(all_X.shape[0], WINDOW_NUM, -1)
test_X_2d = test_X.reshape(test_X.shape[0], WINDOW_NUM, -1)
train_X_2d, val_X_2d, train_y, val_y = train_test_split(
    all_X_2d, all_y, test_size=0.2, random_state=42, stratify=all_y)

# max-min normalization
# @TODO: 这里的归一化方式可能不太对，需要根据实际情况调整(或统一最大最小值)
train_X_2d = (train_X_2d - np.min(train_X_2d)) / (np.max(train_X_2d) - np.min(train_X_2d))
val_X_2d = (val_X_2d - np.min(val_X_2d)) / (np.max(val_X_2d) - np.min(val_X_2d))
test_X_2d = (test_X_2d - np.min(test_X_2d)) / (np.max(test_X_2d) - np.min(test_X_2d))

# 增加通道维度
train_X_2d = train_X_2d[:, :, :, np.newaxis]
val_X_2d = val_X_2d[:, :, :, np.newaxis]
test_X_2d = test_X_2d[:, :, :, np.newaxis]
print("训练集：", train_X_2d.shape, train_y.shape)
print("验证集：", val_X_2d.shape, val_y.shape)
print("测试集：", test_X_2d.shape, test_y.shape)

# 创建卷积神经网络模型
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(1, WINDOW_NUM), activation='relu', input_shape=(WINDOW_NUM, 18, 1)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 编译模型
# model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.compile(optimizer='adam', loss=keras.losses.BinaryFocalCrossentropy(gamma=2.0, from_logits=False), metrics=['accuracy'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# 训练模型
model.fit(train_X_2d, train_y, epochs=10, batch_size=8, validation_data=(val_X_2d, val_y))

# 对 val_X 进行预测
val_y_pred = model.predict(val_X_2d)
val_y_pred = (val_y_pred > 0.5).astype(int).reshape(-1)
evaluate_model(val_y, val_y_pred)

# 对 test_X 进行预测
test_y_pred = model.predict(test_X_2d)
test_y_pred = (test_y_pred > 0.5).astype(int).reshape(-1)
evaluate_model(test_y, test_y_pred)
